In [ ]:
# Wizualizacja Danych, Lab 7 - Jan Banot
# Zadanie 1: Przygotowanie danych i podstawowa wizualizacja
# Aplikacja Dash z wykresem GDP per capita dla wybranych kontynentów

from dash import Dash, dcc, html, Input, Output
import plotly.express as px

df = px.data.gapminder()

app = Dash(__name__)

continents = df["continent"].unique()

# Layout aplikacji
app.layout = html.Div(
    [
        html.H1("GDP per Capita według kontynentów", style={"textAlign": "center"}),
        html.Div(
            [
                html.Label("Wybierz kontynent:"),
                dcc.Dropdown(
                    id="continent-dropdown",
                    options=[{"label": c, "value": c} for c in continents],
                    value=["Asia", "Americas"],
                    multi=True,
                    placeholder="Wybierz kontynent...",
                ),
            ],
            style={"width": "50%", "margin": "auto", "padding": "20px"},
        ),
        dcc.Graph(id="gdp-graph"),
    ]
)


# Callback do aktualizacji wykresu
@app.callback(Output("gdp-graph", "figure"), Input("continent-dropdown", "value"))
def update_graph(selected_continents):
    if not selected_continents:
        selected_continents = []

    # Filtrowanie danych
    filtered_df = df[df["continent"].isin(selected_continents)]

    # Tworzenie wykresu
    fig = px.box(
        filtered_df,
        x="continent",
        y="gdpPercap",
        color="continent",
        title="GDP per Capita dla wybranych kontynentów",
        labels={"gdpPercap": "GDP per Capita", "continent": "Kontynent"},
    )

    fig.update_layout(
        xaxis_title="Kontynent", yaxis_title="GDP per Capita (USD)", showlegend=True
    )

    return fig


if __name__ == "__main__":
    app.run(debug=True, port=8050)

In [ ]:
# Zadanie 2: Interaktywność — filtracja danych
# Aplikacja Dash z dodatkowymi filtrami: suwak lat, dropdown kraju/kontynentu

df = px.data.gapminder()

app = Dash(__name__)

# Lista dostępnych kontynentów i krajów
continents = df["continent"].unique()
countries = df["country"].unique()
years = sorted(df["year"].unique())

# Layout aplikacji
app.layout = html.Div(
    [
        html.H1("Analiza GDP per Capita", style={"textAlign": "center"}),
        # Sekcja filtrów
        html.Div(
            [
                # Dropdown dla kontynentu
                html.Div(
                    [
                        html.Label("Wybierz kontynent:"),
                        dcc.Dropdown(
                            id="continent-dropdown",
                            options=[{"label": c, "value": c} for c in continents],
                            value=["Asia", "Europe"],
                            multi=True,
                            placeholder="Wybierz kontynent...",
                        ),
                    ],
                    style={
                        "width": "45%",
                        "display": "inline-block",
                        "padding": "10px",
                    },
                ),
                # Dropdown dla kraju
                html.Div(
                    [
                        html.Label("Wybierz kraj (opcjonalnie):"),
                        dcc.Dropdown(
                            id="country-dropdown",
                            options=[{"label": c, "value": c} for c in countries],
                            value=[],
                            multi=True,
                            placeholder="Wybierz kraj...",
                        ),
                    ],
                    style={
                        "width": "45%",
                        "display": "inline-block",
                        "padding": "10px",
                    },
                ),
            ],
            style={"width": "80%", "margin": "auto"},
        ),
        # Suwak dla zakresu lat
        html.Div(
            [
                html.Label("Wybierz zakres lat:"),
                dcc.RangeSlider(
                    id="year-slider",
                    min=min(years),
                    max=max(years),
                    step=5,
                    value=[min(years), max(years)],
                    marks={str(year): str(year) for year in years},
                    tooltip={"placement": "bottom", "always_visible": True},
                ),
            ],
            style={"width": "80%", "margin": "auto", "padding": "20px"},
        ),
        # Wybór typu wykresu
        html.Div(
            [
                html.Label("Typ wykresu:"),
                dcc.RadioItems(
                    id="chart-type",
                    options=[
                        {"label": "Wykres liniowy", "value": "line"},
                        {"label": "Wykres słupkowy", "value": "bar"},
                    ],
                    value="line",
                    inline=True,
                ),
            ],
            style={"width": "80%", "margin": "auto", "padding": "10px"},
        ),
        # Wykres
        dcc.Graph(id="gdp-graph"),
    ]
)


# Callback do aktualizacji dropdown krajów na podstawie wybranego kontynentu
@app.callback(
    Output("country-dropdown", "options"), Input("continent-dropdown", "value")
)
def update_country_options(selected_continents):
    if not selected_continents:
        return [{"label": c, "value": c} for c in countries]

    filtered_countries = df[df["continent"].isin(selected_continents)][
        "country"
    ].unique()
    return [{"label": c, "value": c} for c in sorted(filtered_countries)]


# Funkcja filtrująca dane
def filter_data(df, continents, countries, year_range):
    """Filtruje dane na podstawie wybranych kryteriów."""
    filtered_df = df.copy()

    # Filtrowanie po zakresie lat
    filtered_df = filtered_df[
        (filtered_df["year"] >= year_range[0]) & (filtered_df["year"] <= year_range[1])
    ]

    # Jeśli wybrano konkretne kraje, użyj ich
    if countries and len(countries) > 0:
        filtered_df = filtered_df[filtered_df["country"].isin(countries)]
    # W przeciwnym razie filtruj po kontynentach
    elif continents and len(continents) > 0:
        filtered_df = filtered_df[filtered_df["continent"].isin(continents)]

    return filtered_df


# Callback do aktualizacji wykresu
@app.callback(
    Output("gdp-graph", "figure"),
    [
        Input("continent-dropdown", "value"),
        Input("country-dropdown", "value"),
        Input("year-slider", "value"),
        Input("chart-type", "value"),
    ],
)
def update_graph(selected_continents, selected_countries, year_range, chart_type):
    # Filtrowanie danych
    filtered_df = filter_data(df, selected_continents, selected_countries, year_range)

    if filtered_df.empty:
        return px.scatter(title="Brak danych do wyświetlenia")

    # Określenie koloru i grupy
    if selected_countries and len(selected_countries) > 0:
        color_by = "country"
        title = f"GDP per Capita dla wybranych krajów ({year_range[0]}-{year_range[1]})"
    else:
        color_by = "continent"
        title = f"GDP per Capita dla wybranych kontynentów ({year_range[0]}-{year_range[1]})"

    # Tworzenie wykresu w zależności od typu
    if chart_type == "line":
        # Agregacja danych dla wykresu liniowego
        if color_by == "continent":
            agg_df = (
                filtered_df.groupby(["year", "continent"])["gdpPercap"]
                .mean()
                .reset_index()
            )
            fig = px.line(
                agg_df,
                x="year",
                y="gdpPercap",
                color="continent",
                title=title,
                labels={"gdpPercap": "Średnie GDP per Capita", "year": "Rok"},
                markers=True,
            )
        else:
            fig = px.line(
                filtered_df,
                x="year",
                y="gdpPercap",
                color="country",
                title=title,
                labels={"gdpPercap": "GDP per Capita", "year": "Rok"},
                markers=True,
            )
    else:  # bar chart
        # Agregacja danych dla wykresu słupkowego
        if color_by == "continent":
            agg_df = (
                filtered_df.groupby(["year", "continent"])["gdpPercap"]
                .mean()
                .reset_index()
            )
            fig = px.bar(
                agg_df,
                x="year",
                y="gdpPercap",
                color="continent",
                title=title,
                labels={"gdpPercap": "Średnie GDP per Capita", "year": "Rok"},
                barmode="group",
            )
        else:
            fig = px.bar(
                filtered_df,
                x="year",
                y="gdpPercap",
                color="country",
                title=title,
                labels={"gdpPercap": "GDP per Capita", "year": "Rok"},
                barmode="group",
            )

    fig.update_layout(
        xaxis_title="Rok",
        yaxis_title="GDP per Capita (USD)",
        legend_title=color_by.capitalize(),
    )

    return fig


if __name__ == "__main__":
    app.run(debug=True, port=8051)

In [ ]:
# Zadanie 3: Rozbudowa dashboardu — wiele wykresów i elementów interaktywnych
# Dashboard z dwoma wykresami: GDP per Capita i Life Expectancy

from dash import Dash, dcc, html, Input, Output
import plotly.express as px

df = px.data.gapminder()

app = Dash(__name__)

# Lista dostępnych kontynentów i lat
continents = df["continent"].unique()
years = sorted(df["year"].unique())

# Layout aplikacji
app.layout = html.Div(
    [
        html.H1(
            "Dashboard: PKB i Długość Życia",
            style={"textAlign": "center", "color": "#2c3e50"},
        ),
        # Sekcja filtrów
        html.Div(
            [
                # Dropdown dla kontynentu
                html.Div(
                    [
                        html.Label("Wybierz kontynent:", style={"fontWeight": "bold"}),
                        dcc.Dropdown(
                            id="continent-dropdown",
                            options=[{"label": c, "value": c} for c in continents],
                            value=["Europe", "Asia", "Americas"],
                            multi=True,
                            placeholder="Wybierz kontynent...",
                        ),
                    ],
                    style={"width": "60%", "margin": "auto", "padding": "10px"},
                ),
                # Suwak dla zakresu lat
                html.Div(
                    [
                        html.Label("Wybierz zakres lat:", style={"fontWeight": "bold"}),
                        dcc.RangeSlider(
                            id="year-slider",
                            min=min(years),
                            max=max(years),
                            step=5,
                            value=[min(years), max(years)],
                            marks={str(year): str(year) for year in years},
                            tooltip={"placement": "bottom", "always_visible": True},
                        ),
                    ],
                    style={"width": "80%", "margin": "auto", "padding": "20px"},
                ),
            ],
            style={
                "backgroundColor": "#ecf0f1",
                "padding": "20px",
                "borderRadius": "10px",
                "margin": "20px",
            },
        ),
        # Kontener na wykresy - dwa obok siebie
        html.Div(
            [
                # Wykres 1: GDP per Capita
                html.Div(
                    [
                        html.H3(
                            "PKB per Capita w czasie", style={"textAlign": "center"}
                        ),
                        dcc.Graph(id="gdp-graph", style={"height": "400px"}),
                    ],
                    style={"width": "48%"},
                ),
                # Wykres 2: Life Expectancy
                html.Div(
                    [
                        html.H3(
                            "Długość życia w czasie", style={"textAlign": "center"}
                        ),
                        dcc.Graph(id="life-exp-graph", style={"height": "400px"}),
                    ],
                    style={"width": "48%"},
                ),
            ],
            style={
                "display": "flex",
                "justifyContent": "space-around",
                "margin": "20px",
            },
        ),
    ]
)


# Callback do aktualizacji obu wykresów jednocześnie
@app.callback(
    [Output("gdp-graph", "figure"), Output("life-exp-graph", "figure")],
    [Input("continent-dropdown", "value"), Input("year-slider", "value")],
)
def update_graphs(selected_continents, year_range):
    # Obsługa pustego wyboru
    if not selected_continents:
        selected_continents = []

    # Filtrowanie danych
    filtered_df = df[
        (df["continent"].isin(selected_continents))
        & (df["year"] >= year_range[0])
        & (df["year"] <= year_range[1])
    ]

    # Agregacja danych - średnia dla każdego kontynentu i roku
    agg_df = (
        filtered_df.groupby(["year", "continent"])
        .agg({"gdpPercap": "mean", "lifeExp": "mean"})
        .reset_index()
    )

    # Wykres 1: GDP per Capita
    if filtered_df.empty:
        fig_gdp = px.scatter(title="Brak danych do wyświetlenia")
    else:
        fig_gdp = px.line(
            agg_df,
            x="year",
            y="gdpPercap",
            color="continent",
            title=f"Średnie PKB per Capita ({year_range[0]}-{year_range[1]})",
            labels={
                "gdpPercap": "PKB per Capita (USD)",
                "year": "Rok",
                "continent": "Kontynent",
            },
            markers=True,
        )
        fig_gdp.update_layout(
            xaxis_title="Rok",
            yaxis_title="PKB per Capita (USD)",
            legend_title="Kontynent",
            hovermode="x unified",
            height=400,
            margin=dict(l=50, r=50, t=50, b=50),
        )

    # Wykres 2: Life Expectancy
    if filtered_df.empty:
        fig_life = px.scatter(title="Brak danych do wyświetlenia")
    else:
        fig_life = px.line(
            agg_df,
            x="year",
            y="lifeExp",
            color="continent",
            title=f"Średnia długość życia ({year_range[0]}-{year_range[1]})",
            labels={
                "lifeExp": "Długość życia (lata)",
                "year": "Rok",
                "continent": "Kontynent",
            },
            markers=True,
        )
        fig_life.update_layout(
            xaxis_title="Rok",
            yaxis_title="Długość życia (lata)",
            legend_title="Kontynent",
            hovermode="x unified",
            height=400,
            margin=dict(l=50, r=50, t=50, b=50),
        )

    return fig_gdp, fig_life


if __name__ == "__main__":
    app.run(debug=True, port=8052)

In [ ]:
# Zadanie 4: Wizualizacja własnych danych / API
# Dashboard z tabelą danych i wykresem słupkowym - dane sprzedaży

from dash import Dash, dcc, html, Input, Output, dash_table
import plotly.express as px
import pandas as pd

# Przykładoww dane sprzedaży
sales_data = {
    "Miesiąc": [
        "Styczeń",
        "Luty",
        "Marzec",
        "Kwiecień",
        "Maj",
        "Czerwiec",
        "Lipiec",
        "Sierpień",
        "Wrzesień",
        "Październik",
        "Listopad",
        "Grudzień",
    ],
    "Sprzedaż_Elektronika": [
        45000,
        52000,
        48000,
        61000,
        55000,
        67000,
        72000,
        69000,
        58000,
        63000,
        78000,
        95000,
    ],
    "Sprzedaż_Odzież": [
        32000,
        28000,
        35000,
        42000,
        38000,
        45000,
        41000,
        39000,
        47000,
        52000,
        68000,
        82000,
    ],
    "Sprzedaż_AGD": [
        18000,
        21000,
        19000,
        24000,
        22000,
        28000,
        25000,
        27000,
        23000,
        26000,
        35000,
        42000,
    ],
    "Liczba_Klientów": [
        1200,
        1350,
        1280,
        1520,
        1450,
        1680,
        1750,
        1690,
        1480,
        1590,
        1920,
        2350,
    ],
    "Region": [
        "Północ",
        "Południe",
        "Wschód",
        "Zachód",
        "Północ",
        "Południe",
        "Wschód",
        "Zachód",
        "Północ",
        "Południe",
        "Wschód",
        "Zachód",
    ],
}

df = pd.DataFrame(sales_data)

app = Dash(__name__)

categories = ["Sprzedaż_Elektronika", "Sprzedaż_Odzież", "Sprzedaż_AGD"]

# Layout aplikacji
app.layout = html.Div(
    [
        html.H1(
            "Dashboard Sprzedaży", style={"textAlign": "center", "color": "#2c3e50"}
        ),
        # Sekcja filtrów
        html.Div(
            [
                html.Div(
                    [
                        html.Label(
                            "Wybierz kategorię produktów:", style={"fontWeight": "bold"}
                        ),
                        dcc.Dropdown(
                            id="category-dropdown",
                            options=[
                                {"label": cat.replace("_", " "), "value": cat}
                                for cat in categories
                            ],
                            value=categories,
                            multi=True,
                            placeholder="Wybierz kategorię...",
                        ),
                    ],
                    style={
                        "width": "45%",
                        "display": "inline-block",
                        "padding": "10px",
                    },
                ),
                html.Div(
                    [
                        html.Label("Wybierz region:", style={"fontWeight": "bold"}),
                        dcc.Dropdown(
                            id="region-dropdown",
                            options=[
                                {"label": r, "value": r} for r in df["Region"].unique()
                            ],
                            value=df["Region"].unique().tolist(),
                            multi=True,
                            placeholder="Wybierz region...",
                        ),
                    ],
                    style={
                        "width": "45%",
                        "display": "inline-block",
                        "padding": "10px",
                    },
                ),
            ],
            style={
                "width": "90%",
                "margin": "auto",
                "backgroundColor": "#ecf0f1",
                "padding": "20px",
                "borderRadius": "10px",
                "marginBottom": "20px",
            },
        ),
        # Tabela danych
        html.Div(
            [
                html.H3("Tabela danych sprzedaży", style={"textAlign": "center"}),
                dash_table.DataTable(
                    id="sales-table",
                    columns=[{"name": col, "id": col} for col in df.columns],
                    data=df.to_dict("records"),
                    style_table={"overflowX": "auto"},
                    style_cell={
                        "textAlign": "center",
                        "padding": "10px",
                        "minWidth": "100px",
                    },
                    style_header={
                        "backgroundColor": "#3498db",
                        "color": "white",
                        "fontWeight": "bold",
                    },
                    style_data_conditional=[
                        {"if": {"row_index": "odd"}, "backgroundColor": "#f8f9fa"}
                    ],
                    page_size=6,
                    sort_action="native",
                    filter_action="native",
                ),
            ],
            style={"width": "90%", "margin": "auto", "marginBottom": "20px"},
        ),
        # Wykres słupkowy
        html.Div(
            [
                html.H3("Wykres sprzedaży", style={"textAlign": "center"}),
                dcc.Graph(id="sales-bar-chart", style={"height": "450px"}),
            ],
            style={"width": "90%", "margin": "auto"},
        ),
        # Podsumowanie
        html.Div(
            id="summary-stats",
            style={
                "width": "90%",
                "margin": "auto",
                "padding": "20px",
                "marginTop": "20px",
                "backgroundColor": "#e8f4f8",
                "borderRadius": "10px",
            },
        ),
    ]
)


# Callback do aktualizacji tabeli i wykresu
@app.callback(
    [
        Output("sales-table", "data"),
        Output("sales-bar-chart", "figure"),
        Output("summary-stats", "children"),
    ],
    [Input("category-dropdown", "value"), Input("region-dropdown", "value")],
)
def update_dashboard(selected_categories, selected_regions):
    # Filtrowanie danych po regionie
    if not selected_regions:
        selected_regions = []
    filtered_df = df[df["Region"].isin(selected_regions)]

    # Aktualizacja tabeli
    table_data = filtered_df.to_dict("records")

    # Przygotowanie danych do wykresu (format długi)
    if not selected_categories:
        selected_categories = []

    chart_data = []
    for _, row in filtered_df.iterrows():
        for cat in selected_categories:
            chart_data.append(
                {
                    "Miesiąc": row["Miesiąc"],
                    "Kategoria": cat.replace("Sprzedaż_", ""),
                    "Wartość": row[cat],
                    "Region": row["Region"],
                }
            )

    chart_df = pd.DataFrame(chart_data)

    # Tworzenie wykresu słupkowego
    if chart_df.empty:
        fig = px.bar(title="Brak danych do wyświetlenia")
    else:
        fig = px.bar(
            chart_df,
            x="Miesiąc",
            y="Wartość",
            color="Kategoria",
            barmode="group",
            title="Sprzedaż według kategorii produktów",
            labels={"Wartość": "Wartość sprzedaży (PLN)", "Miesiąc": "Miesiąc"},
            color_discrete_sequence=px.colors.qualitative.Set2,
        )
        fig.update_layout(
            xaxis_title="Miesiąc",
            yaxis_title="Wartość sprzedaży (PLN)",
            legend_title="Kategoria",
            height=450,
            margin=dict(l=50, r=50, t=50, b=50),
            xaxis={
                "categoryorder": "array",
                "categoryarray": [
                    "Styczeń",
                    "Luty",
                    "Marzec",
                    "Kwiecień",
                    "Maj",
                    "Czerwiec",
                    "Lipiec",
                    "Sierpień",
                    "Wrzesień",
                    "Październik",
                    "Listopad",
                    "Grudzień",
                ],
            },
        )

    # Podsumowanie statystyk
    if filtered_df.empty:
        summary = html.P("Brak danych do podsumowania")
    else:
        total_sales = (
            sum(filtered_df[cat].sum() for cat in selected_categories)
            if selected_categories
            else 0
        )
        total_customers = filtered_df["Liczba_Klientów"].sum()
        avg_sales = total_sales / len(filtered_df) if len(filtered_df) > 0 else 0

        summary = html.Div(
            [
                html.H4(
                    "Podsumowanie", style={"textAlign": "center", "color": "#2c3e50"}
                ),
                html.Div(
                    [
                        html.Div(
                            [
                                html.H5("Łączna sprzedaż"),
                                html.P(
                                    f"{total_sales:,.0f} PLN",
                                    style={"fontSize": "24px", "color": "#27ae60"},
                                ),
                            ],
                            style={
                                "width": "30%",
                                "display": "inline-block",
                                "textAlign": "center",
                            },
                        ),
                        html.Div(
                            [
                                html.H5("Średnia sprzedaż/miesiąc"),
                                html.P(
                                    f"{avg_sales:,.0f} PLN",
                                    style={"fontSize": "24px", "color": "#3498db"},
                                ),
                            ],
                            style={
                                "width": "30%",
                                "display": "inline-block",
                                "textAlign": "center",
                            },
                        ),
                        html.Div(
                            [
                                html.H5("Łączna liczba klientów"),
                                html.P(
                                    f"{total_customers:,}",
                                    style={"fontSize": "24px", "color": "#e74c3c"},
                                ),
                            ],
                            style={
                                "width": "30%",
                                "display": "inline-block",
                                "textAlign": "center",
                            },
                        ),
                    ]
                ),
            ]
        )

    return table_data, fig, summary


if __name__ == "__main__":
    app.run(debug=True, port=8053)

In [ ]:
# Zadanie 5: Aktualizacja danych i odświeżanie w czasie rzeczywistym
# Dashboard z dcc.Interval - symulacja danych w czasie rzeczywistym

from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import pandas as pd
import math
from datetime import datetime

app = Dash(__name__)

app.layout = html.Div(
    [
        html.H1(
            "Monitor Danych w Czasie Rzeczywistym",
            style={"textAlign": "center", "color": "#2c3e50"},
        ),
        # Kontrolki
        html.Div(
            [
                html.Div(
                    [
                        html.Label(
                            "Interwał odświeżania:", style={"fontWeight": "bold"}
                        ),
                        dcc.Dropdown(
                            id="interval-dropdown",
                            options=[
                                {"label": "1 sekunda", "value": 1000},
                                {"label": "2 sekundy", "value": 2000},
                                {"label": "5 sekund", "value": 5000},
                            ],
                            value=2000,
                            clearable=False,
                        ),
                    ],
                    style={
                        "width": "200px",
                        "display": "inline-block",
                        "marginRight": "20px",
                    },
                ),
                html.Div(
                    [
                        html.Label("Status:", style={"fontWeight": "bold"}),
                        html.Div(
                            id="status-indicator",
                            style={"color": "green", "fontSize": "18px"},
                        ),
                    ],
                    style={"display": "inline-block"},
                ),
            ],
            style={
                "padding": "20px",
                "backgroundColor": "#ecf0f1",
                "borderRadius": "10px",
                "margin": "20px",
            },
        ),
        # Komponent Interval - odświeża co 2 sekundy
        dcc.Interval(id="interval-component", interval=2000, n_intervals=0),
        # Liczniki - karty z aktualnymi wartościami
        html.Div(
            [
                html.Div(
                    [
                        html.H4("🌡️ Temperatura"),
                        html.H2(id="temp-value"),
                    ],
                    style={
                        "width": "30%",
                        "display": "inline-block",
                        "textAlign": "center",
                        "backgroundColor": "#e74c3c",
                        "color": "white",
                        "padding": "20px",
                        "borderRadius": "10px",
                        "margin": "10px",
                    },
                ),
                html.Div(
                    [
                        html.H4("💧 Wilgotność"),
                        html.H2(id="humidity-value"),
                    ],
                    style={
                        "width": "30%",
                        "display": "inline-block",
                        "textAlign": "center",
                        "backgroundColor": "#3498db",
                        "color": "white",
                        "padding": "20px",
                        "borderRadius": "10px",
                        "margin": "10px",
                    },
                ),
                html.Div(
                    [
                        html.H4("🔵 Ciśnienie"),
                        html.H2(id="pressure-value"),
                    ],
                    style={
                        "width": "30%",
                        "display": "inline-block",
                        "textAlign": "center",
                        "backgroundColor": "#27ae60",
                        "color": "white",
                        "padding": "20px",
                        "borderRadius": "10px",
                        "margin": "10px",
                    },
                ),
            ],
            style={"textAlign": "center", "margin": "20px"},
        ),
        # Wykres w czasie rzeczywistym
        html.Div(
            [
                html.H3(
                    "Wykres danych w czasie rzeczywistym", style={"textAlign": "center"}
                ),
                dcc.Graph(id="realtime-graph"),
            ],
            style={"width": "90%", "margin": "auto"},
        ),
        # Licznik aktualizacji
        html.Div(
            [
                html.P(
                    id="update-counter",
                    style={"textAlign": "center", "color": "#7f8c8d"},
                )
            ],
            style={"marginTop": "20px"},
        ),
    ]
)


# Callback do aktualizacji interwału
@app.callback(
    Output("interval-component", "interval"), Input("interval-dropdown", "value")
)
def update_interval(interval_value):
    return interval_value


# Callback do aktualizacji wykresu
@app.callback(
    Output("realtime-graph", "figure"),
    Input("interval-component", "n_intervals"),
)
def update_graph(n_intervals):
    # Generowanie danych
    data = []
    for i in range(30):
        idx = n_intervals - 29 + i
        if idx < 0:
            idx = 0
        temp = 20.0 + 3.0 * math.sin(idx / 10.0)
        hum = 50.0 + 10.0 * math.cos(idx / 15.0)
        data.append({"Punkt": i + 1, "Temperatura": temp, "Wilgotność": hum})

    df = pd.DataFrame(data)

    # Tworzenie wykresu z Plotly Express
    fig = px.line(
        df,
        x="Punkt",
        y=["Temperatura", "Wilgotność"],
        title=f"Dane w czasie rzeczywistym (aktualizacja #{n_intervals})",
        markers=True,
    )

    fig.update_layout(
        height=400,
        xaxis_title="Punkt pomiarowy",
        yaxis_title="Wartość",
        legend_title="Pomiar",
    )

    return fig


# Callbacki do aktualizacji wartości
@app.callback(
    Output("temp-value", "children"), Input("interval-component", "n_intervals")
)
def update_temp(n):
    val = 20.0 + 3.0 * math.sin(n / 10.0)
    return f"{val:.1f}°C"


@app.callback(
    Output("humidity-value", "children"), Input("interval-component", "n_intervals")
)
def update_humidity(n):
    val = 50.0 + 10.0 * math.cos(n / 15.0)
    return f"{val:.1f}%"


@app.callback(
    Output("pressure-value", "children"), Input("interval-component", "n_intervals")
)
def update_pressure(n):
    val = 1013.0 + 2.0 * math.sin(n / 20.0)
    return f"{val:.1f} hPa"


@app.callback(
    Output("update-counter", "children"), Input("interval-component", "n_intervals")
)
def update_counter(n):
    return f"Liczba aktualizacji: {n}"


@app.callback(
    Output("status-indicator", "children"), Input("interval-component", "n_intervals")
)
def update_status(n):
    now = datetime.now().strftime("%H:%M:%S")
    return f"● Aktywny ({now})"


if __name__ == "__main__":
    app.run(debug=True, port=8054)

In [ ]:
# Zadanie 6: Ekspozycja na dużą ilość danych – wydajność i limitacje
# Dashboard demonstrujący różne strategie optymalizacji dla dużych zbiorów danych

from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import time

# Generowanie dużej próbki danych (100 000 punktów)
np.random.seed(42)
n_points = 100_000

# Symulacja danych czasowych z trendem i szumem
x = np.arange(n_points)
trend = 50 + 0.001 * x  # Powolny trend wzrostowy
seasonal = 10 * np.sin(2 * np.pi * x / 1000)  # Sezonowość
noise = np.random.normal(0, 5, n_points)  # Szum
y = trend + seasonal + noise

# Druga seria danych
y2 = (
    30 + 0.0005 * x + 8 * np.cos(2 * np.pi * x / 800) + np.random.normal(0, 3, n_points)
)

df_full = pd.DataFrame({"Indeks": x, "Seria_A": y, "Seria_B": y2})

print(f"Wygenerowano {n_points:,} punktów danych")
print(
    f"Rozmiar DataFrame: {df_full.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB"
)

app = Dash(__name__)

# Strategie optymalizacji
optimization_strategies = [
    {"label": "Wszystkie dane (100k)", "value": "all"},
    {"label": "Co 10 punkt (10k)", "value": "every_10"},
    {"label": "Co 100 punkt (1k)", "value": "every_100"},
    {"label": "Agregacja średnia (1k okien)", "value": "agg_mean"},
    {"label": "Agregacja min/max (widełki)", "value": "agg_minmax"},
    {"label": "Losowe próbkowanie (10k)", "value": "random_sample"},
]

app.layout = html.Div(
    [
        html.H1(
            "Wydajność wizualizacji dużych zbiorów danych",
            style={"textAlign": "center", "color": "#2c3e50"},
        ),
        html.Div(
            [
                html.P(
                    [
                        html.Strong("Zbiór danych: "),
                        f"{n_points:,} punktów pomiarowych (2 serie czasowe)",
                    ]
                ),
            ],
            style={"textAlign": "center", "marginBottom": "20px"},
        ),
        # Panel kontrolny
        html.Div(
            [
                html.Div(
                    [
                        html.Label(
                            "Strategia optymalizacji:", style={"fontWeight": "bold"}
                        ),
                        dcc.Dropdown(
                            id="strategy-dropdown",
                            options=optimization_strategies,
                            value="every_100",
                            clearable=False,
                        ),
                    ],
                    style={
                        "width": "300px",
                        "display": "inline-block",
                        "marginRight": "30px",
                    },
                ),
                html.Div(
                    [
                        html.Label("Zakres danych:", style={"fontWeight": "bold"}),
                        dcc.RangeSlider(
                            id="range-slider",
                            min=0,
                            max=n_points,
                            step=1000,
                            value=[0, n_points],
                            marks={
                                i: f"{i // 1000}k"
                                for i in range(0, n_points + 1, 20000)
                            },
                            tooltip={"placement": "bottom", "always_visible": True},
                        ),
                    ],
                    style={"width": "500px", "display": "inline-block"},
                ),
            ],
            style={
                "padding": "20px",
                "backgroundColor": "#ecf0f1",
                "borderRadius": "10px",
                "margin": "20px",
            },
        ),
        # Informacje o wydajności
        html.Div(
            id="performance-info",
            style={
                "padding": "15px",
                "margin": "20px",
                "backgroundColor": "#fff3cd",
                "borderRadius": "10px",
                "border": "1px solid #ffc107",
            },
        ),
        # Wykres główny
        html.Div(
            [
                dcc.Graph(id="main-chart", style={"height": "500px"}),
            ],
            style={"width": "95%", "margin": "auto"},
        ),
        # Porównanie strategii
        html.Div(
            [
                html.H3(
                    "Porównanie strategii optymalizacji", style={"textAlign": "center"}
                ),
                html.Div(
                    id="strategy-comparison",
                    style={
                        "display": "flex",
                        "justifyContent": "space-around",
                        "flexWrap": "wrap",
                        "margin": "20px",
                    },
                ),
            ]
        ),
        # Opis strategii
        html.Div(
            [
                html.H4("Opis strategii optymalizacji:", style={"color": "#2c3e50"}),
                html.Ul(
                    [
                        html.Li(
                            [
                                html.Strong("Wszystkie dane: "),
                                "Wyświetla każdy punkt - najwolniejsze, ale najprecyzyjniejsze",
                            ]
                        ),
                        html.Li(
                            [
                                html.Strong("Co N punkt: "),
                                "Próbkowanie równomierne - szybkie, zachowuje ogólny kształt",
                            ]
                        ),
                        html.Li(
                            [
                                html.Strong("Agregacja średnia: "),
                                "Dzieli dane na okna i liczy średnią - wygładza dane",
                            ]
                        ),
                        html.Li(
                            [
                                html.Strong("Agregacja min/max: "),
                                "Pokazuje zakres wartości w każdym oknie - zachowuje ekstrema",
                            ]
                        ),
                        html.Li(
                            [
                                html.Strong("Losowe próbkowanie: "),
                                "Losowy wybór punktów - może gubić wzorce",
                            ]
                        ),
                    ]
                ),
            ],
            style={
                "padding": "20px",
                "margin": "20px",
                "backgroundColor": "#e8f4f8",
                "borderRadius": "10px",
            },
        ),
    ]
)


def apply_optimization(df, strategy, start_idx, end_idx):
    """Stosuje wybraną strategię optymalizacji do danych."""
    # Filtrowanie zakresu
    df_range = df[(df["Indeks"] >= start_idx) & (df["Indeks"] < end_idx)].copy()
    n_original = len(df_range)

    start_time = time.time()

    if strategy == "all":
        result_df = df_range
        description = f"Wszystkie {n_original:,} punktów"

    elif strategy == "every_10":
        result_df = df_range.iloc[::10]
        description = f"Co 10 punkt: {len(result_df):,} z {n_original:,}"

    elif strategy == "every_100":
        result_df = df_range.iloc[::100]
        description = f"Co 100 punkt: {len(result_df):,} z {n_original:,}"

    elif strategy == "agg_mean":
        # Agregacja przez średnią w oknach
        window_size = max(1, n_original // 1000)
        df_range["window"] = df_range["Indeks"] // window_size
        result_df = (
            df_range.groupby("window")
            .agg({"Indeks": "mean", "Seria_A": "mean", "Seria_B": "mean"})
            .reset_index(drop=True)
        )
        description = (
            f"Średnia w oknach ({window_size} pkt): {len(result_df):,} z {n_original:,}"
        )

    elif strategy == "agg_minmax":
        # Agregacja min/max dla zachowania ekstremów
        window_size = max(1, n_original // 500)
        df_range["window"] = df_range["Indeks"] // window_size
        agg_min = df_range.groupby("window").agg(
            {"Indeks": "min", "Seria_A": "min", "Seria_B": "min"}
        )
        agg_max = df_range.groupby("window").agg(
            {"Indeks": "max", "Seria_A": "max", "Seria_B": "max"}
        )
        # Przeplatamy min i max
        result_list = []
        for w in agg_min.index:
            result_list.append(agg_min.loc[w])
            result_list.append(agg_max.loc[w])
        result_df = pd.DataFrame(result_list).reset_index(drop=True)
        description = (
            f"Min/Max w oknach ({window_size} pkt): {len(result_df):,} z {n_original:,}"
        )

    elif strategy == "random_sample":
        sample_size = min(10000, n_original)
        result_df = df_range.sample(n=sample_size, random_state=42).sort_values(
            "Indeks"
        )
        description = f"Losowe próbkowanie: {len(result_df):,} z {n_original:,}"

    else:
        result_df = df_range
        description = "Nieznana strategia"

    processing_time = (time.time() - start_time) * 1000  # w ms

    return result_df, description, len(result_df), processing_time


@app.callback(
    [Output("main-chart", "figure"), Output("performance-info", "children")],
    [Input("strategy-dropdown", "value"), Input("range-slider", "value")],
)
def update_chart(strategy, range_values):
    start_idx, end_idx = range_values

    # Pomiar czasu przetwarzania
    total_start = time.time()

    # Zastosowanie optymalizacji
    df_optimized, description, n_points_display, proc_time = apply_optimization(
        df_full, strategy, start_idx, end_idx
    )

    # Tworzenie wykresu
    fig = go.Figure()

    fig.add_trace(
        go.Scattergl(  # Scattergl dla lepszej wydajności
            x=df_optimized["Indeks"],
            y=df_optimized["Seria_A"],
            mode="lines",
            name="Seria A",
            line=dict(color="#e74c3c", width=1),
        )
    )

    fig.add_trace(
        go.Scattergl(
            x=df_optimized["Indeks"],
            y=df_optimized["Seria_B"],
            mode="lines",
            name="Seria B",
            line=dict(color="#3498db", width=1),
        )
    )

    fig.update_layout(
        title=f"Wizualizacja danych - {description}",
        xaxis_title="Indeks",
        yaxis_title="Wartość",
        hovermode="x unified",
        height=500,
        legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
    )

    total_time = (time.time() - total_start) * 1000

    # Informacje o wydajności
    performance_info = html.Div(
        [
            html.H4("📊 Informacje o wydajności", style={"margin": "0 0 10px 0"}),
            html.Div(
                [
                    html.Div(
                        [html.Strong("Strategia: "), description],
                        style={"marginBottom": "5px"},
                    ),
                    html.Div(
                        [
                            html.Strong("Punktów do wyświetlenia: "),
                            f"{n_points_display:,}",
                            html.Span(
                                f" (redukcja: {(1 - n_points_display / (end_idx - start_idx)) * 100:.1f}%)"
                                if n_points_display < (end_idx - start_idx)
                                else "",
                                style={"color": "#27ae60"},
                            ),
                        ],
                        style={"marginBottom": "5px"},
                    ),
                    html.Div(
                        [html.Strong("Czas przetwarzania: "), f"{proc_time:.1f} ms"],
                        style={"marginBottom": "5px"},
                    ),
                    html.Div(
                        [
                            html.Strong("Całkowity czas aktualizacji: "),
                            f"{total_time:.1f} ms",
                        ]
                    ),
                ]
            ),
        ]
    )

    return fig, performance_info


@app.callback(Output("strategy-comparison", "children"), Input("range-slider", "value"))
def update_comparison(range_values):
    """Generuje porównanie wszystkich strategii."""
    start_idx, end_idx = range_values

    cards = []
    strategies = ["all", "every_10", "every_100", "agg_mean", "random_sample"]
    strategy_names = {
        "all": "Wszystkie",
        "every_10": "Co 10",
        "every_100": "Co 100",
        "agg_mean": "Średnia",
        "random_sample": "Losowe",
    }

    for strategy in strategies:
        _, _, n_points, proc_time = apply_optimization(
            df_full, strategy, start_idx, end_idx
        )

        # Kolor karty w zależności od wydajności
        if proc_time < 10:
            bg_color = "#d4edda"  # Zielony - szybko
        elif proc_time < 50:
            bg_color = "#fff3cd"  # Żółty - średnio
        else:
            bg_color = "#f8d7da"  # Czerwony - wolno

        card = html.Div(
            [
                html.H5(strategy_names[strategy], style={"margin": "5px 0"}),
                html.P(
                    f"Punkty: {n_points:,}",
                    style={"margin": "3px 0", "fontSize": "14px"},
                ),
                html.P(
                    f"Czas: {proc_time:.1f} ms",
                    style={"margin": "3px 0", "fontSize": "14px"},
                ),
            ],
            style={
                "backgroundColor": bg_color,
                "padding": "15px",
                "borderRadius": "8px",
                "margin": "5px",
                "minWidth": "120px",
                "textAlign": "center",
            },
        )

        cards.append(card)

    return cards


if __name__ == "__main__":
    app.run(debug=True, port=8055)

In [ ]:
# Zadanie 7: Export i podzielenie danych do PDF, PNG
# Dashboard z funkcją eksportu wykresów do różnych formatów

from dash import Dash, dcc, html, Input, Output, State, callback_context
import plotly.express as px
import os

# Katalog do zapisywania wykresów
EXPORT_DIR = "exports"
os.makedirs(EXPORT_DIR, exist_ok=True)

df = px.data.gapminder()

app = Dash(__name__)

continents = df["continent"].unique()
years = sorted(df["year"].unique())

# Layout aplikacji
app.layout = html.Div(
    [
        html.H1(
            "Eksport wykresów do PNG i PDF",
            style={"textAlign": "center", "color": "#2c3e50"},
        ),
        # Panel kontrolny
        html.Div(
            [
                # Filtry danych
                html.Div(
                    [
                        html.Div(
                            [
                                html.Label("Kontynent:", style={"fontWeight": "bold"}),
                                dcc.Dropdown(
                                    id="continent-dropdown",
                                    options=[
                                        {"label": c, "value": c} for c in continents
                                    ],
                                    value=["Europe", "Asia", "Americas"],
                                    multi=True,
                                ),
                            ],
                            style={
                                "width": "250px",
                                "display": "inline-block",
                                "marginRight": "20px",
                            },
                        ),
                        html.Div(
                            [
                                html.Label("Rok:", style={"fontWeight": "bold"}),
                                dcc.Dropdown(
                                    id="year-dropdown",
                                    options=[
                                        {"label": str(y), "value": y} for y in years
                                    ],
                                    value=2007,
                                    clearable=False,
                                ),
                            ],
                            style={
                                "width": "120px",
                                "display": "inline-block",
                                "marginRight": "20px",
                            },
                        ),
                        html.Div(
                            [
                                html.Label(
                                    "Typ wykresu:", style={"fontWeight": "bold"}
                                ),
                                dcc.RadioItems(
                                    id="chart-type",
                                    options=[
                                        {"label": "Scatter", "value": "scatter"},
                                        {"label": "Bar", "value": "bar"},
                                        {"label": "Box", "value": "box"},
                                    ],
                                    value="scatter",
                                    inline=True,
                                ),
                            ],
                            style={"display": "inline-block"},
                        ),
                    ]
                ),
            ],
            style={
                "padding": "20px",
                "backgroundColor": "#ecf0f1",
                "borderRadius": "10px",
                "margin": "20px",
            },
        ),
        # Przyciski eksportu
        html.Div(
            [
                html.H4(
                    "Eksport wykresu:",
                    style={"display": "inline-block", "marginRight": "20px"},
                ),
                html.Button(
                    "📷 Eksport PNG",
                    id="btn-export-png",
                    style={
                        "backgroundColor": "#3498db",
                        "color": "white",
                        "border": "none",
                        "padding": "10px 20px",
                        "borderRadius": "5px",
                        "cursor": "pointer",
                        "marginRight": "10px",
                        "fontSize": "14px",
                    },
                ),
                html.Button(
                    "📄 Eksport PDF",
                    id="btn-export-pdf",
                    style={
                        "backgroundColor": "#e74c3c",
                        "color": "white",
                        "border": "none",
                        "padding": "10px 20px",
                        "borderRadius": "5px",
                        "cursor": "pointer",
                        "marginRight": "10px",
                        "fontSize": "14px",
                    },
                ),
            ],
            style={
                "padding": "20px",
                "margin": "20px",
                "backgroundColor": "#f8f9fa",
                "borderRadius": "10px",
                "textAlign": "center",
            },
        ),
        # Status eksportu
        html.Div(
            id="export-status",
            style={
                "padding": "15px",
                "margin": "20px",
                "textAlign": "center",
                "fontSize": "16px",
            },
        ),
        # Wykres
        html.Div(
            [
                dcc.Graph(
                    id="main-chart",
                    style={"height": "500px"},
                    config={
                        # Konfiguracja paska narzędzi z opcjami eksportu
                        "toImageButtonOptions": {
                            "format": "png",  # domyślny format
                            "filename": "wykres_dash",
                            "height": 600,
                            "width": 1000,
                            "scale": 2,  # wyższa jakość
                        },
                        "displaylogo": False,
                        "modeBarButtonsToAdd": [
                            "drawline",
                            "drawopenpath",
                            "eraseshape",
                        ],
                    },
                ),
            ],
            style={"width": "90%", "margin": "auto"},
        ),
        # Store do przechowywania danych wykresu
        dcc.Store(id="chart-data-store"),
        # Komponenty do pobierania plików
        dcc.Download(id="download-image"),
        # Lista wyeksportowanych plików
        html.Div(
            [
                html.H4("📁 Wyeksportowane pliki:", style={"color": "#2c3e50"}),
                html.Div(id="exported-files-list"),
            ],
            style={
                "padding": "20px",
                "margin": "20px",
                "backgroundColor": "#fff3cd",
                "borderRadius": "10px",
            },
        ),
    ]
)


# Callback do aktualizacji wykresu
@app.callback(
    Output("main-chart", "figure"),
    [
        Input("continent-dropdown", "value"),
        Input("year-dropdown", "value"),
        Input("chart-type", "value"),
    ],
)
def update_chart(selected_continents, selected_year, chart_type):
    if not selected_continents:
        selected_continents = []

    # Filtrowanie danych
    filtered_df = df[
        (df["continent"].isin(selected_continents)) & (df["year"] == selected_year)
    ]

    # Tworzenie wykresu w zależności od typu
    if chart_type == "scatter":
        fig = px.scatter(
            filtered_df,
            x="gdpPercap",
            y="lifeExp",
            size="pop",
            color="continent",
            hover_name="country",
            title=f"PKB per capita vs Długość życia ({selected_year})",
            labels={
                "gdpPercap": "PKB per capita (USD)",
                "lifeExp": "Długość życia (lata)",
                "pop": "Populacja",
            },
            size_max=60,
        )
    elif chart_type == "bar":
        # Top 10 krajów wg GDP
        top_countries = filtered_df.nlargest(10, "gdpPercap")
        fig = px.bar(
            top_countries,
            x="country",
            y="gdpPercap",
            color="continent",
            title=f"Top 10 krajów wg PKB per capita ({selected_year})",
            labels={"gdpPercap": "PKB per capita (USD)", "country": "Kraj"},
        )
        fig.update_xaxes(tickangle=45)
    else:  # box
        fig = px.box(
            filtered_df,
            x="continent",
            y="gdpPercap",
            color="continent",
            title=f"Rozkład PKB per capita ({selected_year})",
            labels={"gdpPercap": "PKB per capita (USD)", "continent": "Kontynent"},
        )

    fig.update_layout(
        height=500,
        margin=dict(l=50, r=50, t=80, b=50),
    )

    return fig


# Callback do eksportu
@app.callback(
    [Output("export-status", "children"), Output("exported-files-list", "children")],
    [
        Input("btn-export-png", "n_clicks"),
        Input("btn-export-pdf", "n_clicks"),
    ],
    [
        State("main-chart", "figure"),
        State("continent-dropdown", "value"),
        State("year-dropdown", "value"),
        State("chart-type", "value"),
    ],
    prevent_initial_call=True,
)
def export_chart(n_png, n_pdf, n_svg, n_html, figure, continents, year, chart_type):
    ctx = callback_context
    if not ctx.triggered:
        return "", list_exported_files()

    button_id = ctx.triggered[0]["prop_id"].split(".")[0]

    # Tworzenie obiektu Figure z danych
    fig = go.Figure(figure)

    # Generowanie nazwy pliku
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    base_filename = f"wykres_{chart_type}_{year}_{timestamp}"

    try:
        if button_id == "btn-export-png":
            filepath = os.path.join(EXPORT_DIR, f"{base_filename}.png")
            fig.write_image(filepath, width=1200, height=700, scale=2)
            status = html.Div(
                [
                    html.Span("✅ ", style={"color": "green"}),
                    f"Wykres zapisany jako PNG: {filepath}",
                ],
                style={"color": "green"},
            )

        elif button_id == "btn-export-pdf":
            filepath = os.path.join(EXPORT_DIR, f"{base_filename}.pdf")
            fig.write_image(filepath, width=1200, height=700)
            status = html.Div(
                [
                    html.Span("✅ ", style={"color": "green"}),
                    f"Wykres zapisany jako PDF: {filepath}",
                ],
                style={"color": "green"},
            )

        else:
            status = ""

    except Exception as e:
        error_msg = str(e)
        if "kaleido" in error_msg.lower():
            status = html.Div(
                [
                    html.Span("⚠️ ", style={"color": "orange"}),
                    "Eksport obrazów wymaga biblioteki kaleido. ",
                    html.Code("pip install kaleido"),
                ],
                style={"color": "orange"},
            )
        else:
            status = html.Div(
                [
                    html.Span("❌ ", style={"color": "red"}),
                    f"Błąd eksportu: {error_msg}",
                ],
                style={"color": "red"},
            )

    return status, list_exported_files()


def list_exported_files():
    """Lista wyeksportowanych plików."""
    if not os.path.exists(EXPORT_DIR):
        return html.P("Brak wyeksportowanych plików")

    files = sorted(os.listdir(EXPORT_DIR), reverse=True)
    if not files:
        return html.P("Brak wyeksportowanych plików")

    file_items = []
    for f in files[:10]:  # Ostatnie 10 plików
        filepath = os.path.join(EXPORT_DIR, f)
        size = os.path.getsize(filepath) / 1024  # KB
        ext = f.split(".")[-1].upper()

        # Ikona w zależności od formatu
        icons = {"PNG": "🖼️", "PDF": "📄", "SVG": "📊", "HTML": "🌐"}
        icon = icons.get(ext, "📁")

        file_items.append(
            html.Li(f"{icon} {f} ({size:.1f} KB)", style={"padding": "5px"})
        )

    return html.Ul(file_items, style={"listStyleType": "none", "padding": "0"})


if __name__ == "__main__":
    app.run(debug=True, port=8056)